# Walk Forward: A Realistic Approach to Backtesting

In [ ]:
???

![](<src/10_Table_Validation Methods.png>)

## Load the data

In [10]:
import pandas as pd

import pandas as pd
df = pd.read_excel('data/microsoft-linkedin-processed.xlsx' , index_col=0)
df.index = pd.to_datetime(df.index)

# df = pd.read_excel('data/Microsoft_LinkedIn_Processed.xlsx', parse_dates=['Date'], index_col=0)
df = df.drop(columns='change_tomorrow_direction')
df

,Close,High,Low,Open,Volume,change_tomorrow
Price,,,,,,
2016-12-01,59.200001,60.150002,58.939999,60.110001,34542100,0.084387
2016-12-02,59.250000,59.470001,58.799999,59.080002,25515700,1.610763
2016-12-05,60.220001,60.590000,59.560001,59.700001,23552700,-0.450376
2016-12-06,59.950001,60.459999,59.799999,60.430000,19907000,2.313831
2016-12-07,61.369999,61.380001,59.799999,60.009998,30809000,-0.590068
...,...,...,...,...,...,...
2024-11-04,408.459991,410.420013,405.570007,409.799988,19672300,0.729111
2024-11-05,411.459991,414.899994,408.079987,408.369995,17626000,2.075301
2024-11-06,420.179993,420.450012,410.519989,412.420013,26681800,1.234046


## Walk Forward Validation

### How `TimeSeriesSplit` works

In [11]:
from sklearn.model_selection import TimeSeriesSplit

In [114]:
ts = TimeSeriesSplit(test_size=200)

In [115]:
list_df_train = []
list_df_test = []

for index_train, index_test in ts.split(df):
    list_df_train.append(df.iloc[index_train])
    list_df_test.append(df.iloc[index_test])


## Machine Learning Model

### Separate the data

1. Target: which variable do you want to predict?
2. Explanatory: which variables will you use to calculate the prediction?

In [117]:
y = df.change_tomorrow
X = df[['Open','High','Low','Close','Volume']]

In [ ]:
list_df_train = []
list_df_test = []

for index_train, index_test in ts.split(y):
    X_train, y_train = X.iloc[index_train], y.iloc[index_train]
    X_test, y_test = X.iloc[index_test], y.iloc[index_test]


                  Open        High         Low       Close    Volume
Price                                                               
2020-11-18  213.649994  215.169998  210.929993  211.080002  28372800
2020-11-19  211.380005  213.029999  209.929993  212.419998  24792700
2020-11-20  212.199997  213.289993  210.000000  210.389999  22843100
********************************************************************************
                  Open        High         Low       Close    Volume
Price                                                               
2021-09-07  301.010010  301.089996  298.200012  300.179993  17180400
2021-09-08  299.779999  300.609985  297.470001  300.209991  15046800
2021-09-09  300.820007  302.140015  297.000000  297.250000  19927000
********************************************************************************
                  Open        High         Low       Close    Volume
Price                                                               
2022-06-23

### Simulate one computation of the ML model

- Compute the model
- Calculate predictions on the test set
- Evaluate how good the model is

In [127]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

model_dt = DecisionTreeRegressor(max_depth=15, random_state=42)
model_dt.fit(X_train, y_train)

y_pred = model_dt.predict(X_test)
error_mse = mean_squared_error(y_test, y_pred)
error_mse

1.8405433037825438

In [128]:
for index_train, index_test in ts.split(y):
    X_train, y_train = X.iloc[index_train], y.iloc[index_train]
    X_test, y_test = X.iloc[index_test], y.iloc[index_test]
    model_dt.fit(X_train, y_train)
    y_pred = model_dt.predict(X_test)
    error_mse = mean_squared_error(y_test, y_pred)
    print(error_mse)


37.60386790948279
7.6388686681750775
5.651981628246065
4.638096856714091
1.8405433037825438


### Add the procedure inside the for loop

In [ ]:
model_dt = DecisionTreeRegressor(max_depth=15, random_state=42)

error_mse_list = []

for index_train, index_test in ts.split(df):
    
    ???

## Anchored Walk Forward evaluation in backtesting

![](<src/10_Table_Validation Methods.png>)

### Create a new strategy

In [ ]:
from backtesting import Backtest, Strategy

In [ ]:
bt = Backtest(df, ???, cash=10000, commission=.002, exclusive_orders=True)

In [ ]:
class Regression(Strategy):
    limit_buy = 1
    limit_sell = -5
    
    def init(self):
        self.model = DecisionTreeRegressor(max_depth=15, random_state=42)
        self.already_bought = False
        
        X_train = ???
        y_train = ???
        
        self.model.fit(X=X_train, y=y_train)

    def next(self):
        explanatory_today = self.data.df.iloc[[-1], ???]
        forecast_tomorrow = self.model.predict(explanatory_today)[0]
        
        if forecast_tomorrow > self.limit_buy and self.already_bought == False:
            self.buy()
            self.already_bought = True
        elif forecast_tomorrow < self.limit_sell and self.already_bought == True:
            self.sell()
            self.already_bought = False
        else:
            pass

In [ ]:
class WalkForwardAnchored(Regression):
    def next(self):
        
        # we don't take any action and move on to the following day
        if len(self.data) < ???:
            return
        
        # we retrain the model each x days
        if len(self.data) % ??? == 0:
            X_train = self.data.df.iloc[:, :-1]
            y_train = self.data.df.iloc[:, -1]

            self.model.fit(X_train, y_train)

            super().next()
            
        else:
            
            super().next()

In [ ]:
from backtesting import Backtest
bt = Backtest(df, WalkForwardAnchored, cash=10000, commission=.002, exclusive_orders=True)

In [ ]:
import multiprocessing as mp
mp.set_start_method('fork')

In [ ]:
stats_skopt, heatmap, optimize_result = bt.optimize(
    limit_buy = range(0, 6), limit_sell = range(-6, 0),
    maximize='Return [%]',
    max_tries=500,
    random_state=42,
    return_heatmap=True,
    return_optimization=True,
    method='skopt'
    )

dff = heatmap.reset_index()
dff = dff.sort_values('Return [%]', ascending=False)
dff

## Unanchored Walk Forward

### Create a library of strategies

`strategies.py`

### Create the unanchored walk forward class

![](<src/10_Table_Validation Methods.png>)

### Import the strategy and perform the backtest

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import strategies

In [ ]:
strategies.WalkForwardUnanchored

In [ ]:
bt_unanchored = Backtest(df, strategies.WalkForwardUnanchored, cash=10000, commission=.002, exclusive_orders=True)

stats_skopt, heatmap, optimize_result = bt_unanchored.optimize(
    limit_buy = range(0, 6), limit_sell = range(-6, 0),
    maximize='Return [%]',
    max_tries=500,
    random_state=42,
    return_heatmap=True,
    return_optimization=True,
    method='skopt'
    )

dff = heatmap.reset_index()
dff = dff.sort_values('Return [%]', ascending=False)
dff

### Interpret the strategies' performance

- Both anchored and unanchored backtesting

## Practice to master the knowledge

Work on the challenge with another dataset:

1. Learn the <a>mental models</a> to solve the challenge faster.
2. Complete the <a href="10C_Walk Forward Regression.ipynb">notebook</a>.